### Example 4: Computing layer optical properties and writing a simple forward model
Part of ESE 156 Remote Sensing Class;
Christian Frankenberg
__________
So far, we have learned about the basics of optical properties of trace gases and their dependence on pressure and temperature in particular (both in terms of the line-shapes but also redistribution of line-strengths within a band due to the Boltzmann distribution of the population of rotational states).

The purpose of this exercise is to compute optical properties of a vertically structured atmosphere (varying in pressure, temperature and of course mixing ratios). We will first look at idealized examples, for which we assume a direct light-path between the sun and an observer on the ground (e.g. a TCCON measurement taken right in the Linde Robinson building!).

You will also learn a little about accessing Numerical Weather Forecast models, see e.g. for accessing grib data:
http://nbviewer.jupyter.org/gist/jswhit/8635665

### Layer properties

If the cross sections vary substantially as a function of pressure and temperature (as they do for rotational/vibrational transitions), we need to divide the atmosphere into separate layers to compute optical properties for each of those, assuming a specific p and T for this layer. 

What we also need is the vertical column density (VCD) of each layer separately. The total VCD of a gas $i$ with volume mixing ratio in dry air $VMR_i$ is given as 
$$VCD = \int_{z_s}^\infty VMR(z) \cdot \rho_{dryAir}(z) dz\,,$$
where we use the number density $\rho_{N}$ (e.g. in molec/cm$^3$) and $z_s$ being the surface altitude.

This quantity depends on both surface pressure as well as the actual $VMR$. For well-mixed gases, where changes in surface pressure outweigh the impact of changes in $VMR$, we often use the column-averaged mixing ration $X_i$:
$$X_{Gas} = \frac{VCD_{Gas}}{\int_{z_s}^\infty \rho_{dryAir}(z) dz}$$



To compute layer properties, we just integrate from one pressure level to the next:

$$VCD_i = \int_{z_i}^{z_{i+1}} VMR(z) \cdot \rho_{N,dryAir}(z) dz = \bar{VMR}\frac{\Delta p_{dry}}{Mg}$$

where we use molecular mass M (kg/molec) for the air layer, which depends on humidity to some degree. It is interesting to note that any impact of T or z is not apparent, can you derive this equation? 

Now we just need to approximate $g$, which we can either assume as constant or really compute as function of latitude $\phi$ (geoid instead of perfect sphere) as well as height.

$$g(\phi,0m) \approx 9.780327 m/s^2 \cdot(1+0.0053024 \sin^2 \phi - 0.0000058 \sin^2 2\phi)$$

This equation is the International Gravity Formula 1967 (see https://en.wikipedia.org/wiki/Gravity_of_Earth#Latitude_model). 

The gravity at the equator is 9.780327$\,$m/s$^2$ and at the poles 9.83218620588$\,$m/s$^2$, a change of about 0.5%. This is small but non-negligible if you want to get column averaged trace gas concentrations to better than 1ppm for CO$_2$, which is only 0.25% of the background of 400ppm!

The last part if gravity dependence on height as the current values is given for the sea level equivalent.

In general, gravity from a body of mass $m$ (here using Earth, with $m_{earth}=5.9722\cdot 10^{24}kg$ is
$$g_o=\frac{G m_{Earth}}{r^2_{Earth}}\,$$
with G being the gravitational constant and $r_{Earth}$ the reference Earth radius of 6.371$\cdot 10^6$m.

The ratio of the gravity at sea level and at height h is thus
$$g_h/g_0 = \frac{r_{earth}^2}{(r_{earth}+h)^2}$$

We can thus define 
$$g(\phi,h) = g(\phi,0m)\cdot \frac{r_{earth}^2}{(r_{earth}+h)^2}$$
##### Few more constants
gas constant $R_d$ in J/kg/K for dry air: 
R_d = 287.04;

universal gas constant: 
R_universal = 8.314472;

Avogadro number: 
Na = 6.0221415e23;

_____


### Actually calculating the layer sub-column densities based on atmospheric profile data

Now that we have learned the basics of how to compute layer properties, we need to apply that knowledge using some real atmospheric data. For the lack of a better example (a lot of other real-time data is in a funny GRIB format), I chose some data from the openDAP server from NOAA, available as netCDF4 datasets. In this last example here, I actually pre-downloaded data from the MERRA re-analysis (large file!).

In [ ]:
using NCDatasets
using ProgressMeter
# from mpl_toolkits.basemap import Basemap
#from urllib.request import urlopen

In [ ]:
mydate="20181007"
startTime = "18z" # There is 00, 06, 12 and 18 in UTC, i.e. 6 hourly data stacked together

# What latitude do we want? Take Caltech as example
myLat = 34.1377
myLon = -118.1253

# Try to access numerical weather forecast data (GFS here, using openDAP datasets)
#url = 'http://nomads.ncep.noaa.gov:9090/dods/gfs_0p25/gfs' + mydate + '/gfs_0p25_'+startTime
#url = 'https://goldsmr3.sci.gsfc.nasa.gov:443/opendap/MERRA/MAI6NVANA.5.2.0/2015/06/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf'
url = "../files/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4"
if !isfile(url)
    download("ftp://fluo.gps.caltech.edu/XYZT_ESE156/Data/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4", "url")
end
# decided to download on my own (large!)
# Dowload file here: ftp://fluo.gps.caltech.edu/XYZT_ESE156/Data/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4
url = "../files/MERRA300.prod.assim.inst6_3d_ana_Nv.20150613.hdf.nc4"

In [ ]:
ds = Dataset(url)

# See how easy it is to actually extract data? Note the [:] in the end reads in ALL the data in one step
lat   = ds["YDim"][:]
lon   = ds["XDim"][:]
# Temperature profile
T     = ds["T"][:]
# specific humidity profile
q     = ds["QV"][:]
# mean pressure profile:
p     = ds["Height"][:]
# Surafce pressure
psurf = ds["PS"][:]
# Time in UTC
time  = ds["TIME"][:]

# AK and BK global attributes (important to calculate pressure half-levels)
ak = ds.attrib["HDF_GLOBAL.ak"][:]
bk = ds.attrib["HDF_GLOBAL.bk"][:]

@show ds["T"]

close(ds)

_____
Somewhat off-topic but now that we have read global data, I just wanted to show you how you can display them on the globe. Usually, I am using python matplotlib and basemap and am still struggling to do something similar in Julia (you can use PyPlot and PyCall though!).
_____

In [ ]:
using Plots
heatmap(lon, lat,psurf[:,:,1]' ./ 100.0 , clim = (800,1000))
title!("Surface Pressure [hPa]")
ylims!(-60,78)

In [ ]:
# Find the indices in lat/lon that best match Caltech here:
iLat = argmin(abs.(lat .- myLat))
iLon = argmin(abs.(lon .- myLon))

@show iLat, iLon
@show psurf[iLat,iLon,:] ./ 100.

In [ ]:
plot( T[iLat,iLon,:,1],p, yaxis=:flip, lw=2, label=" 0-UTC")
plot!(T[iLat,iLon,:,2],p, yaxis=:flip, lw=2, label=" 6-UTC")
plot!(T[iLat,iLon,:,3],p, yaxis=:flip, lw=2, label="12-UTC")
plot!(T[iLat,iLon,:,4],p, yaxis=:flip, lw=2, label="18-UTC")
xlabel!("Temperature [K]")
ylabel!("Pressure [hPa]")
title!("Temperature profile at Caltech (6-hourly data on 20150613)")

In [ ]:
plot( T[iLat,iLon,:,1],p, yaxis=(:log,:flip), lw=2, label=" 0-UTC")
plot!(T[iLat,iLon,:,2],p, yaxis=(:log,:flip), lw=2, label=" 6-UTC")
plot!(T[iLat,iLon,:,3],p, yaxis=(:log,:flip), lw=2, label="12-UTC")
plot!(T[iLat,iLon,:,4],p, yaxis=(:log,:flip), lw=2, label="18-UTC")
xlabel!("Temperature [K]")
ylabel!("Pressure [hPa]")
title!("Temperature profile at Caltech (6-hourly data on 20150613)")

Here, the y-axis is logarithmic. What does this roughly represent in a linear way?

Can you identify the Tropopause, Stratosphere, Stratopause? 

What is the tropopause height?

In [ ]:
plot( q[iLat,iLon,:,1],p, yaxis=:flip, lw=2, label=" 0-UTC")
plot!(q[iLat,iLon,:,2],p, yaxis=:flip, lw=2, label=" 6-UTC")
plot!(q[iLat,iLon,:,3],p, yaxis=:flip, lw=2, label="12-UTC")
plot!(q[iLat,iLon,:,4],p, yaxis=:flip, lw=2, label="18-UTC")
xlabel!("Specific humidity [kg/kg]")
ylabel!("Pressure [hPa]")
title!("Specific humidity profile at Caltech (6-hourly data on 20150613)")

#### Computing the vertical sub-columns now per layer:
One tricky part in numerical weather forecast data is how levels are defined, see Section 2.2 in http://www.ecmwf.int/sites/default/files/elibrary/2015/9210-part-iii-dynamics-and-numerical-procedures.pdf

Usually, variables are defined in $NLEV$ layers, with pressures at the so-called half-levels defined at the boundaries between the layers, providing $NLEV+1$ half-levels:
$$p_{k+1/2} = A_{k+1/2} + B_{k+1/2}p_{surf}$$




---
Now we would like to get a function that extracts the layer properties within an atmospheric profile as a function of time slice as well as lat/lon. Below, we do this naively, in reality, we would write a function for this, which can also take gravity changes into account (also easier to use these in for other planets then!).

In [ ]:
# Let's choose one index in the time-domain 
index = 1

# Surface pressure at Caltech and time slice 1 (scalar):
ps_local = psurf[iLon, iLat,index]
# q and T profiles at Caltech and time slice 1 (vector):
q_local  = q[iLon,iLat,:,index]
T_local  = T[iLon,iLat,:,index]

# Half-level (at the boundaries, one index more), 
# in Pa 
p_half = (ak + bk*ps_local)
p_full = (p_half[2:end] + p_half[1:end-1])/2
n_layers = length(T_local)

# Let us ignore gravity changes for simplicity
g₀ = 9.8196 # (m/s^2) 
Na = 6.0221415e23;

dryMass = 28.9647e-3  /Na  # in kg/molec, weighted average for N2 and O2
wetMass = 18.01528e-3 /Na  # just H2O

ratio = dryMass/wetMass
    

# also get a VMR vector of H2O (volumetric!)
vmr_h2o = zeros(n_layers,)
vcd_dry = zeros(n_layers,)
vcd_h2o = zeros(n_layers,)

# Now actually compute the layer VCDs
for i=1:n_layers 
    Δp = p_half[i+1] - p_half[i]
    vmr_h2o[i] = q_local[i] * ratio
    vmr_dry = 1 - vmr_h2o[i]
    M  = vmr_dry * dryMass + vmr_h2o[i] * wetMass
    vcd_dry[i] = vmr_dry*Δp/(M*g₀*100.0^2)   #includes m2->cm2
    vcd_h2o[i] = vmr_h2o[i]*Δp/(M*g₀*100^2)
end

@show sum(vcd_dry)

## Computing cross sections of trace gases per layer
The next step would be to compute the actual cross sections per atmospheric layer and then multiply these with the actual VCD per layer for the different trace gases. As we now have the dry air VCDs per layer, we just need to multiply this vector with a VMR profile vector for each trace gas to get to the gas VCD (we already have this for H$_2$O here). Then, we need to compute the cross sections as function of pressure and temperature based on the layer properties.

For computing the cross sections, we will use the HITRAN (high-resolution transmission molecular absorption) database and make use of their new python interface HAPI (http://hitran.org/hapi/). Essentially, HITRAN is a compilation of spectroscopic parameters needed to compute cross sections from first principles. An excellent short description is provided in http://hitran.org/docs/definitions-and-units/. Most important paremeters to keep in mind for now are 
<li> $\nu_{ij}$, the wavenumber of the spectral line transition (cm$^{-1}$) in vacuum
<li> $S_{ij}$, the spectral line intensity (cm$^{−1}$/(molecule$\cdot$ cm$^{−2}$) at Tref=296K
<li> $\gamma_{γair}$, the air-broadened half width at half maximum (HWHM) (cm$^{−1}$/atm) at Tref=296K and reference pressure p$_{ref}$=1atm
<li> $E″$, the lower-state energy of the transition (cm$^{-1}$)

Using this database, we need to first convert $S_{ij}$ at the reference temperature to the actual temperature (Boltzmann distribution, lower state energy important here). Then, we need to convolve the $S_{ij}$(T) with the line-shape function $f(\nu-\nu_{ij},T,p)$, which itself is pressure and temperature dependent (pressure impacts air-broadening and temperature the doppler broadening as well as air-broadening coefficients). 

The absorption coefficient $k(\nu)$ is then simply:
$$k_{ij}(\nu,T,p)=S_{ij}(T)\cdot f(\nu;nu_{ij},T,p)$$

_____




##### Using the new Julia RadiativeTransfer tools
Luckily, we can now rely on our Julia interface to do most of the work for reading the spectral database and actually computing the line-shape as well as all the temperature corrections, partition sums and so forth (all of this used to be very tedious based on Fortran-style ASCII files as database). 

In [ ]:
using RadiativeTransfer.CrossSection

Please dowload these files into your files folder (database for H2O, CH4 and CO2):
ftp://fluo.gps.caltech.edu/XYZT_hitran/hitran_molec_id_1_H2O.par
ftp://fluo.gps.caltech.edu/XYZT_hitran/hitran_molec_id_6_CH4.par
ftp://fluo.gps.caltech.edu/XYZT_hitran/hitran_molec_id_2_CO2.par

In [ ]:
# Let us try to get some data around the 1.5-1.7µm spectral range for H2O, CO2 and CH4
# Minimum wavenumber
ν_min  = 6100.0
# Maximum wavenumber
ν_xmax = 6400.0

# (we have to know the HITRAN molecule numbers, given in http://hitran.org/docs/molec-meta/)
# Read in HITRAN tables
co2_par = CrossSection.read_hitran("../files/hitran_molec_id_2_CO2.par", mol=2, iso=1, ν_min=ν_min, ν_max=ν_xmax);
ch4_par = CrossSection.read_hitran("../files/hitran_molec_id_6_CH4.par", mol=6, iso=1, ν_min=ν_min, ν_max=ν_xmax);
h2o_par = CrossSection.read_hitran("../files/hitran_molec_id_1_H2O.par", mol=1, iso=1, ν_min=ν_min, ν_max=ν_xmax);

In [ ]:
?co2_par

In [ ]:
# and then plot them:
scatter( h2o_par.νᵢ, h2o_par.Sᵢ, label="H2O",  yaxis=:log, markersize = 2, markeralpha = 0.4)
scatter!(co2_par.νᵢ, co2_par.Sᵢ, label="CO2",  yaxis=:log, markersize = 2, markeralpha = 0.4)
scatter!(ch4_par.νᵢ, ch4_par.Sᵢ, label="CH4",  yaxis=:log, markersize = 2, markeralpha = 0.4)
#plt.semilogy(nu_CH4, sw_CH4, '.',label='CH$_4$', alpha=0.5)
#plt.semilogy(nu_CO2, sw_CO2, '.',label='CO$_2$', alpha=0.5)
#plt.xlim((xmin,xmax))
#plt.xlabel(r'$\nu$ (cm$^{-1}$)')
#plt.ylabel('Line intensity (cm$^{2}$/molec cm$^{-1}$) ')
#plt.legend(loc=0)
#plt.title('Spectral lines of main isotopologues of CO$_2$, CH$_4$ and H$_2$O in the 1.6$\mu$m range')

This looks pretty messy as it includes a lot of very weak transitions, which may not matter after all. However, you can start seeing actual bands appearing with their P,Q and R branched (e.g. the methane 2$\nu_3$ band, an overtone of the $\nu_3$ band) centered at around 6000cm$^{-1}$. 

#### Computing line-shapes and cross sections using RadiativeTransfer.CrossSection
Now we actually want to compute some line-shapes using our tool, which provides options for Doppler, Lorentz, Voigt (more advanced line-shapes in the future). 

In [ ]:
co2_voigt   = make_hitran_model(co2_par, Voigt(), wing_cutoff=10)
h2o_voigt   = make_hitran_model(h2o_par, Voigt(), wing_cutoff=10)
ch4_voigt   = make_hitran_model(ch4_par, Voigt(), wing_cutoff=10)

In [ ]:
# Define wavenumber grid:
ν = 6100:0.01:6400
# Compute cross sections:
σ_co2 = absorption_cross_section(co2_voigt, ν, 1013.0, 296.0);
σ_ch4 = absorption_cross_section(ch4_voigt, ν, 1013.0, 296.0);
σ_h2o = absorption_cross_section(h2o_voigt, ν, 1013.0, 296.0);

In [ ]:
ff = 1e20
plot( ν, ff*σ_h2o,label="H2O", alpha=0.75, yaxis=:log)
plot!(ν, ff*σ_ch4,label="CH4", alpha=0.75, yaxis=:log)
plot!(ν, ff*σ_co2,label="CO2", alpha=0.75, yaxis=:log)
#plt.legend(loc=0)
xlabel!("ν (cm⁻¹)")
ylabel!("σ (/1e20 cm⁻²/molec)")
title!("Cross sections at standard conditions of CO2, CH4 and H2O")

#### Looping through all atmospheric layers!
Now, we want to compute all cross sections for the atmospheric profile obtained for Caltech from the model data. For this, we will actually use a Voigt line-shape now, i.e. a combination or Doppler and pressure broadening.

In [ ]:
# Create matrix of cross sections for each atmospheric layer (takes quite some time!!):
# This 
cs_matrix_co2 = zeros((length(ν),n_layers))
cs_matrix_ch4 = zeros((length(ν),n_layers))
cs_matrix_h2o = zeros((length(ν),n_layers))

# Loop over each layer 
@showprogress for i=1:n_layers
    p_ = p_full[i] / 100 # in hPa
    T_ = T_local[i]
    cs_matrix_co2[:,i] = absorption_cross_section(co2_voigt, ν, p_, T_);
    cs_matrix_ch4[:,i] = absorption_cross_section(ch4_voigt, ν, p_, T_);
    cs_matrix_h2o[:,i] = absorption_cross_section(h2o_voigt, ν, p_, T_);
end

In [ ]:
plot(ν,ff*cs_matrix_co2[:,1:10:end])
xlims!((6170,6270))
xlabel!("ν (cm⁻¹)")
ylabel!("σ (/1e20 cm⁻²/molec)")
title!("Cross sections per 10th layer")

In [ ]:
plot(ν,ff*cs_matrix_co2[:,1:10:end])
xlims!((6205.2,6205.5))
xlabel!("ν (cm⁻¹)")
ylabel!("σ (/1e20 cm⁻²/molec)")
title!("Zoom on a single CO2 line at different layers (every 10th)")

In this figure you can already see that the spectral sampling interval of 0.01cm$^{-1}$ can be somewhat coarse for purely Doppler-broadenend lines higher up in the atmosphere.
_____

#### Generating a synthetic measurement using an up-looking geometry
Now we can start building our own forward model, using the layer-based optical properties and assuming a specific solar zenith angle and a direct light-path between the sun and the actual measurement on the ground, similar to TCCON.

In [ ]:
SZA = 45.0
# Single air mass factor (1 way path), ignores refraction and Earth's curvature here
AMF = 1/cosd(SZA)
vmr_co2 = 400e-6
vmr_ch4 = 1.8e-6
# Generate matrices of optical thickness per layer now for each gas: 
τ_co2 = cs_matrix_co2 .* vcd_dry' * vmr_co2
τ_ch4 = cs_matrix_ch4 .* vcd_dry' * vmr_ch4
#VCD_h2o = dz*rho_N_h2o
τ_h2o = cs_matrix_h2o .* vcd_h2o'

plot( ν, exp.(-AMF*sum(τ_h2o,dims=2)), alpha=0.75, label="H2O")
plot!(ν, exp.(-AMF*sum(τ_ch4,dims=2)), alpha=0.75, label="CH4")
plot!(ν, exp.(-AMF*sum(τ_co2,dims=2)), alpha=0.75, label="CO2")